In [ ]:
#https://www.kaggle.com/christofhenkel/extract-image-features-from-pretrained-nn

In [1]:
import cv2
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook

In [2]:
#read .csv
breed_labels_path = 'D:\\TheDung\\Research\\DataScience\\petfinder-adoption-prediction\\petfinder-adoption-prediction\\breed_labels.csv'
color_labels_path = 'D:\\TheDung\\Research\\DataScience\\petfinder-adoption-prediction\\petfinder-adoption-prediction\\color_labels.csv'
state_labels_path = 'D:\\TheDung\\Research\\DataScience\\petfinder-adoption-prediction\\petfinder-adoption-prediction\\state_labels.csv'
train_path = 'D:\\TheDung\\Research\\DataScience\\petfinder-adoption-prediction\\petfinder-adoption-prediction\\train\\train.csv'
test_path = 'D:\\TheDung\\Research\\DataScience\\petfinder-adoption-prediction\\petfinder-adoption-prediction\\test\\test.csv'
sub_path = 'D:\\TheDung\\Research\\DataScience\\petfinder-adoption-prediction\\petfinder-adoption-prediction\\test\\sample_submission.csv'
#
breed_labels_df = pd.read_csv(breed_labels_path)
color_labels_df = pd.read_csv(color_labels_path)
state_labels_df = pd.read_csv(state_labels_path)
#
color_labels_df.head(5)

,ColorID,ColorName
0,1,Black
1,2,Brown
2,3,Golden
3,4,Yellow
4,5,Cream


In [4]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3,2


In [5]:
img_size = 256
batch_size = 16

In [7]:
#filter only pet_ids
pet_ids = train_df['PetID'].values

#get total of pets
n_batches = len(pet_ids) // batch_size + 1
n_batches

938

In [8]:
from keras.applications.densenet import preprocess_input, DenseNet121

Using TensorFlow backend.


In [9]:
def resize_to_square(im):
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # new_size should be in (width, height) format
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_im

def load_image(path, pet_id):
    image = cv2.imread(f'{path}{pet_id}-1.jpg')
    new_image = resize_to_square(image)
    new_image = preprocess_input(new_image)
    return new_image

In [ ]:
# can tim hieu them: preprocess_input, DenseNet121 (from keras.applications.densenet)

In [10]:
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K
inp = Input((256,256,3))
backbone = DenseNet121(input_tensor = inp, include_top = False)
x = backbone.output
x = GlobalAveragePooling2D()(x)
x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x)
x = AveragePooling1D(4)(x)
out = Lambda(lambda x: x[:,:,0])(x)

m = Model(inp,out)

29089792/29084464 [==============================] - ETA: 31 - ETA: 5: - ETA: 4: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 48s - ETA: 43 - ETA: 40 - ETA: 40 - ETA: 39 - ETA: 37 - ETA: 36 - ETA: 35 - ETA: 35 - ETA: 34 - ETA: 34 - ETA: 33 - ETA: 32 - ETA: 31 - ETA: 29 - ETA: 28 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 26 - ETA: 27 - ETA: 27 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 35 - ETA: 34 - ETA: 35 - ETA: 34 - ETA: 36 - ETA: 37 - ETA: 38 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 37 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 35 - ETA: 34 - ETA: 33 - ETA: 33 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 31 - ETA: 30 - ETA: 30 - ETA: 30 - ETA: 31 - ETA: 26 - ETA: 26 - ETA: 26 - ETA: 26 - ETA: 26 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 27 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 27 - ETA: 28 - ETA: 2

In [11]:
features = {}
train_images_path = 'D:\\TheDung\\Research\\DataScience\\petfinder-adoption-prediction\\petfinder-adoption-prediction\\train_images\\'
for b in tqdm_notebook(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = pet_ids[start:end]
    batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
    for i,pet_id in enumerate(batch_pets):
        try:
            batch_images[i] = load_image(train_images_path, pet_id)
        except:
            pass
    batch_preds = m.predict(batch_images)
    for i,pet_id in enumerate(batch_pets):
        features[pet_id] = batch_preds[i]

In [12]:
train_feats = pd.DataFrame.from_dict(features, orient='index')

In [13]:
train_feats.to_csv('train_img_features.csv')
train_feats.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
86e1089a3,0.002000,0.167800,0.019715,0.015896,0.068162,0.002216,0.005042,0.004828,0.050760,0.047626,...,0.787699,0.176626,0.575706,1.088628,0.439556,0.520460,1.547071,0.832573,0.599093,0.763348
6296e909a,0.002858,0.107450,0.019916,0.023482,0.174765,0.002297,0.005031,0.006338,0.083378,0.049948,...,0.628259,0.686865,0.564000,0.968190,1.070276,1.545742,0.894409,0.838595,0.468238,0.916672
3422e4906,0.002734,0.072015,0.024455,0.018021,0.154207,0.001946,0.004211,0.001576,0.100046,0.039717,...,0.579116,0.557625,1.131405,0.720513,1.496671,0.870955,1.289683,1.184462,0.465114,0.892826
5842f1ff5,0.002106,0.274519,0.054815,0.013727,0.089969,0.001650,0.005506,0.004295,0.118727,0.034790,...,1.295853,0.326143,0.291669,1.608086,1.119176,1.470889,0.591444,0.832755,0.483021,1.134126
850a43f90,0.002185,0.174022,0.044818,0.016244,0.169775,0.002075,0.004421,0.004157,0.099671,0.034441,...,1.092663,0.669894,0.395784,0.886075,1.219730,1.033966,1.065686,0.304054,0.438069,0.676817
